In [1]:
import pickle
import pandas as pd 
import numpy as np

## predict的姓名，依據該姓名去取前後文，並且換成李家賢，加入tags=0/1，端看是否為黑名單

In [2]:
unpickled_df = pd.read_pickle("train_full_data.pkl")

In [ ]:
unpickled_df

In [23]:
import re
# 找出字串中，屬於該名字patretn的人
def find_all_indexes(input_str, search_str):
    matches = [m.start() for m in re.finditer(search_str, input_str)]
    return matches

#示範
s =unpickled_df.iloc[195].article
print(find_all_indexes(s, '陳淳伍'))

[21, 154, 188, 221, 271, 354, 438, 473, 508, 577]


In [41]:
import numpy as np
import pandas as pd

def generate_article_dataframe(unpickled_df, replace_name="李家賢", context_window=(100, 50, 290)):
    """
    根據輸入的文章資料，擷取每個名字在文章中的上下文片段，並標註是否為黑名單，最後輸出 DataFrame。

    參數:
    ----------
    unpickled_df : pandas.DataFrame
        必須包含以下欄位:
        - article (str): 文章全文
        - all_name (list[str]): 該篇文章中所有名字
        - name_list (list[str]): 該篇文章中的黑名單名字
    replace_name : str, 預設 "李家賢"
        在擷取的文字片段中，將原來的人名替換成的名字。
    context_window : tuple(int, int, int), 預設 (100, 50, 290)
        定義上下文截取範圍:
        - 第一個數字: 左邊最大截取距離 (或 20% 文長取最小)
        - 第二個數字: 左邊最小截取距離 (或 20% 文長取最小)
        - 第三個數字: 右邊截取的長度

    回傳:
    ----------
    pandas.DataFrame
        包含以下欄位:
        - article_index : 該片段屬於第幾篇文章
        - truncated_text : 擷取後的文字片段
        - tag : 是否為黑名單 (1 = 黑名單, 0 = 非黑名單)
        - name : 原始名字
    """

    def find_all_indexes(text, substring):
        """回傳 substring 在 text 中所有出現位置 (list of index)。"""
        start = 0
        indexes = []
        while True:
            idx = text.find(substring, start)
            if idx == -1:
                break
            indexes.append(idx)
            start = idx + 1
        return indexes

    article_truncate = []
    article_index = []
    article_tags = []
    article_tags_name = []

    for i in range(len(unpickled_df)):
        article_len = len(unpickled_df.iloc[i].article)
        article_content = unpickled_df.iloc[i].article
        all_name_list = unpickled_df.iloc[i].all_name

        for j in all_name_list:
            name_index = find_all_indexes(article_content, j)

            for k in name_index:
                # 標註黑名單
                if j in unpickled_df.iloc[i].name_list:
                    article_tags.append(1)
                else:
                    article_tags.append(0)

                article_tags_name.append(j)

                # 擷取上下文片段
                left_start = np.clip(k - min(context_window[0], int(article_len * 0.2)), 0, 999999)
                left_end = np.clip(k - min(context_window[1], int(article_len * 0.2)), 0, 999999) + context_window[2]

                txt = article_content[left_start:left_end].replace(j, replace_name)

                article_truncate.append(txt)
                article_index.append(i)

    df = pd.DataFrame({
        "article_index": article_index,
        "truncated_text": article_truncate,
        "tag": article_tags,
        "name": article_tags_name
    })

    return df
    
df_result = generate_article_dataframe(unpickled_df)
print(df_result.head())


   article_index                                     truncated_text  tag name
0              0  ewater Associates)、AQR 資產管理公司、曼氏集團 (Man Group)...    0  張堯勇
1              0  索羅斯的量子基金，也打敗了股神巴菲特的價值投資。近年來台灣也逐漸重視量化交易，摩根亞太量化交...    0  張堯勇
2              0  如何選股是一門學問。目前全球 60 個主要股市交易所，總市值 70 兆美元，而台灣上市上櫃市...    0  張堯勇
3              0  中美貿易戰為例，台股上市、上櫃指數均是負報酬，不過，摩根量化交易卻是正報酬，而且波動度也比較...    0  張堯勇
4              0  不準發放現金股利、部分保險公司必須儘快舉辦現金增資、以後賣的保單利率必須下降。另外台灣全體商...    0  張堯勇


In [54]:
df_result.drop_duplicates(['truncated_text','name','tag']).reset_index(drop = True, inplace = True)

In [57]:
df_result.to_csv('data/Sentiment_model_data/step2_2_output_train_data_300.csv' , index=0)